In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import json
from helpers import *
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
# Load speakers
speakers = pd.read_parquet('data/speaker_attributes.parquet')

In [4]:
speakers.head()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Q161885, Q30]",[Q6581097],1395141751,None,W000178,"[Q82955, Q189290, Q131512, Q1734662, Q294126, ...",[Q327591],None,Q23,George Washington,"[Q698073, Q697949]",item,[Q682443]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[Q145],[Q6581097],1395737157,[Q7994501],None,"[Q214917, Q28389, Q6625963, Q4853732, Q1884422...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Q31],[Q6581097],1380367296,None,None,"[Q36180, Q40348, Q182436, Q1265807, Q205375, Q...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[Q30],[Q6581097],1395142029,None,None,"[Q82955, Q15982858, Q18814623, Q1028181, Q1408...",[Q29468],None,Q207,George W. Bush,"[Q327959, Q464075, Q3586276, Q4450587]",item,"[Q329646, Q682443, Q33203]"
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Q29],[Q6581097],1391704596,None,None,[Q1028181],None,None,Q297,Diego Velázquez,None,item,None


In [5]:
# Get nationalities (first one)
nationalities = speakers.nationality.apply(lambda nt: nt[0] if nt is not None else None)

In [6]:
# Find unique ones excluding None
unique_nat_qids = np.unique(nationalities[nationalities.values != None])

In [7]:
len(unique_nat_qids)

2704

In [8]:
len(nationalities.values)

9055981

In [67]:
# Get all existing country entities
countries = {}
entity_instanceof = get_wiki_entity('P31')
entity_country = get_wiki_entity('Q6256')

for qid in unique_nat_qids:
    entity = get_wiki_entity(qid)
    for instance in entity.getlist(entity_instanceof):
        if instance == entity_country:
            countries[qid] = entity
            break

In [69]:
[country.label for country in countries.values()]

[m'Gabon',
 m'The Gambia',
 m'Guinea',
 m'Guinea-Bissau',
 m"Côte d'Ivoire",
 m'Cameroon',
 m'Cape Verde',
 m'Lesotho',
 m'Liberia',
 m'Libya',
 m'Madagascar',
 m'Malawi',
 m'Mauritania',
 m'Mauritius',
 m'Morocco',
 m'Mozambique',
 m'Namibia',
 m'Niger',
 m'Nigeria',
 m'Uganda',
 m'Rwanda',
 m'São Tomé and Príncipe',
 m'Senegal',
 m'Seychelles',
 m'Sierra Leone',
 m'Somalia',
 m'Sudan',
 m'Kenya',
 m'Ethiopia',
 m'Ghana',
 m'Imperial China',
 m'Kosovo',
 m'Holy Roman Empire',
 m'Kingdom of Aksum',
 m'France',
 m'United Kingdom',
 m"People's Republic of China",
 m'Brazil',
 m'Russia',
 m'Canada',
 m'Great Britain',
 m'American Samoa',
 m'Northern Mariana Islands',
 m'Japan',
 m'North Vietnam',
 m'South Vietnam',
 m'Germany',
 m'Belarus',
 m'Trucial States',
 m'Iceland',
 m'Central African Empire',
 m'Estonia',
 m'Norway',
 m'Pahlavi Dynasty',
 m'England',
 m'Latvia',
 m'Ukraine',
 m'Aruba',
 m'Czech Republic',
 m'Cisalpine Republic',
 m'Slovakia',
 m'Slovenia',
 m'Moldova',
 m'Romania'

In [72]:
country_qids = list(countries.keys())

In [75]:
with open('data/country_qids.json', 'w') as f:
    f.write(json.dumps({'qids': country_qids}))

<font color='red'>WARNING: LONG EXECUTION!</font>

In [73]:
# Find speakers for those countries
country_speakers = speakers[speakers.nationality.apply(lambda nt: any([cid in nt for cid in country_qids]) if nt is not None else False)]

In [78]:
country_speakers.id.to_csv('data/country_speaker_qids.csv')

In [22]:
# Load data from saved files
country_speaker_qids = pd.read_csv('data/country_speaker_qids.csv')
country_speakers = speakers[speakers.id.isin(country_speaker_qids)]

In [41]:
with open('data/country_qids.json') as f:
    country_qids = json.load(f)['qids']

countries = {qid: get_wiki_entity(qid) for qid in country_qids}

In [32]:
len(country_qids)

230

In [28]:
len(country_speakers)

3164693

In [12]:
with open('data/signi-quote-count-combined_1636253317430.json') as f:
    signi_quote_count_dict = json.load(f)

In [13]:
signi_quote_count_df = pd.Series(signi_quote_count_dict, name='signi_quote_count')

In [14]:
# Merge these country speakers with speakers who have significant number of quotes
signi_country_speakers = country_speakers.merge(signi_quote_count_df, left_on='id', right_index=True, how='inner')

In [15]:
len(signi_country_speakers)

436152

In [16]:
# Number of speakers who have more than 1 nationality
(signi_country_speakers.nationality.apply(lambda nt: len(nt)) > 1).sum()

19083

In [29]:
# Filter out those nationalities that are not legitimate countries
signi_country_speakers['country'] = signi_country_speakers.nationality.apply(lambda nt: [n for n in nt if n in country_qids])

In [30]:
# Number of speakers with more nationalities after filtering
(signi_country_speakers.country.apply(lambda nt: len(nt)) > 1).sum()

10291

In [31]:
# Take the first nationality as a country
signi_country_speakers['country'] = signi_country_speakers.country.apply(lambda c: c[0])

In [33]:
# Speakers grouped by country
speakers_by_country = signi_country_speakers.groupby('country')

In [34]:
speakers_by_country.size()

country
Q1000     35
Q1005    113
Q1006     91
Q1007     24
Q1008    174
        ... 
Q971      48
Q974     179
Q977      11
Q983      12
Q986      49
Length: 219, dtype: int64

In [35]:
speakers_by_country.size().min(), speakers_by_country.size().max()

(1, 146486)

In [36]:
speakers_by_country.count().sort_values(by=['id'], ascending=False)

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion,signi_quote_count
country,,,,,,,,,,,,,,,,
Q30,33081,133039,146486,146344,146486,9646,1488,143867,15668,1285,146486,146486,157,146486,4579,146486
Q145,11741,45831,48328,48247,48328,484,0,47576,2626,273,48328,48328,1681,48328,1130,48328
Q16,3366,21215,23987,23963,23987,220,0,23046,3106,121,23987,23987,800,23987,401,23987
Q408,3888,16578,18281,18248,18281,123,0,17978,2100,64,18281,18281,389,18281,318,18281
Q183,1777,16875,17158,17157,17158,106,0,16795,2626,1338,17158,17158,465,17158,698,17158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q237,1,2,2,2,2,2,0,2,0,1,2,2,0,2,2,2
Q180573,0,2,2,2,2,1,0,2,2,0,2,2,0,2,2,2
Q172640,0,1,1,1,1,0,0,1,0,0,1,1,0,1,0,1


In [37]:
country_agg = speakers_by_country.agg({'id': 'count', 'signi_quote_count': 'sum'})

In [38]:
# Number of countries who have less number of speakers in the dataset
(country_agg.id < 10).sum()

20

In [39]:
(country_agg.signi_quote_count < 10).sum()

4

In [42]:
underrep_countries = list(country_agg[country_agg.id < 5].index)
[countries[c].label for c in underrep_countries]

[m'Imperial China',
 m'North Vietnam',
 m'South Vietnam',
 m'Vatican City',
 m'Republic of Artsakh',
 m'Tibet',
 m'Curaçao',
 m'Niue',
 m'Sahrawi Arab Democratic Republic',
 m'German Empire',
 m'Zaire',
 m'Danish Realm',
 m'Republic of Upper Volta']

In [43]:
underrep_countries_by_quote = list(country_agg[country_agg.signi_quote_count < 30].index)
[countries[c].label for c in underrep_countries_by_quote]

[m'Imperial China',
 m'North Vietnam',
 m'South Vietnam',
 m'Republic of Artsakh',
 m'Tibet',
 m'Zaire',
 m'Danish Realm',
 m'Republic of Upper Volta']

In [44]:
country_agg[country_agg.index.isin(underrep_countries)]

,id,signi_quote_count
country,,
Q12060881,1,1
Q172640,1,1
Q180573,2,10
Q237,2,37501
Q244165,2,17
Q2444884,2,24
Q25279,3,48
Q34020,3,46
Q40362,4,60


In [47]:
signi_country_speakers[signi_country_speakers.signi_quote_count > 30].shape

(121075, 17)

In [55]:
country_agg[country_agg.index == 'Q986']

,id,signi_quote_count
country,,
Q986,49,472


In [69]:
def select_top_speakers(speaker_group, top_k=50):
    speaker_group = speaker_group.sort_values(by=['signi_quote_count'], ascending=False)
    return speaker_group.iloc[:top_k]

selected_country_speakers = speakers_by_country.apply(select_top_speakers).reset_index(drop=True)

In [70]:
selected_country_speakers.sort_values(by=['signi_quote_count'], ascending=False)

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion,signi_quote_count,country
3795,"[Donald John Trump, Donald J. Trump, Trump, Th...",[+1946-06-14T00:00:00Z],[Q30],[Q6581097],1395141850,None,None,"[Q911554, Q557880, Q3427922, Q15980158, Q13152...","[Q29468, Q1897793, Q29552, Q29468, Q29468]",[Q787674],Q22686,Donald Trump,"[Q699872, Q22923830, Q38051523]",item,[Q178169],201293,Q30
5136,"[Namo, Modi, Narendra Bhai, Narendra Damodarda...",[+1950-09-17T00:00:00Z],[Q668],[Q6581097],1395415052,"[Q1282294, Q6889284]",None,"[Q82955, Q36180, Q7019111, Q10429346]",[Q10230],None,Q1058,Narendra Modi,"[Q63988227, Q63988919, Q16841231, Q16251961, Q...",item,[Q9089],76404,Q668
3796,"[Barack Hussein Obama II, Barack Obama II, Bar...",[+1961-08-04T00:00:00Z],[Q30],[Q6581097],1395141963,"[Q49085, Q6935055, Q12826303, Q6392846]",O000167,"[Q82955, Q40348, Q15958642, Q28532974, Q372436]",[Q29552],"[Q1765120, Q1540185]",Q76,Barack Obama,"[Q3586671, Q45578, Q4226, Q4791768, Q17067714,...",item,"[Q23540, Q1062789, Q960252, Q426316]",59650,Q30
3097,"[Jorge Mario Bergoglio, Francisco, Pope Franci...",[+1936-12-17T00:00:00Z],"[Q237, Q414]",[Q6581097],1396054797,[Q1056744],None,"[Q63173086, Q593644, Q104050302, Q36180, Q1234...",None,[Q1233889],Q450675,Francis,None,item,[Q9592],37218,Q237
1812,"[Jürgen Norbert Klopp, Jurgen Klopp, Juergen K...",[+1967-06-16T00:00:00Z],[Q183],[Q6581097],1389758184,None,None,"[Q937857, Q628099]",None,None,Q83106,Jürgen Klopp,None,item,[Q75809],33025,Q183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5450,None,[+1994-03-02T00:00:00Z],[Q697],[Q6581097],1363345621,None,None,[Q13381376],None,None,Q26217954,Elson Brechtefeld,None,item,None,1,Q697
5451,None,[+1983-03-31T00:00:00Z],[Q697],[Q6581097],1308221285,None,None,[Q6665249],None,None,Q2105454,Sled Dowabobo,None,item,None,1,Q697
5459,[John Richard Haglelgam],[+1949-08-10T00:00:00Z],[Q702],[Q6581097],1176463720,None,None,[Q82955],None,None,Q448251,John Haglelgam,None,item,None,1,Q702
5460,[Leo Amy Falcam],[+1935-11-20T00:00:00Z],[Q702],[Q6581097],1263701987,None,None,[Q82955],None,None,Q448296,Leo Falcam,None,item,None,1,Q702


In [71]:
selected_country_speakers.groupby('country').count()

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion,signi_quote_count
country,,,,,,,,,,,,,,,,
Q1000,11,33,35,35,35,1,0,35,14,0,35,35,1,35,1,35
Q1005,8,50,50,50,50,1,0,50,11,0,50,50,2,50,5,50
Q1006,18,46,50,50,50,1,0,50,5,0,50,50,1,50,4,50
Q1007,16,24,24,24,24,0,0,24,9,0,24,24,1,24,2,24
Q1008,23,50,50,50,50,0,0,50,8,1,50,50,0,50,9,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q971,20,46,48,48,48,0,0,46,21,0,48,48,5,48,1,48
Q974,23,50,50,50,50,2,0,50,14,1,50,50,2,50,6,50
Q977,2,10,11,11,11,0,0,10,4,1,11,11,0,11,1,11


In [72]:
selected_country_speakers.to_parquet('data/rep_country_speakers.parquet')

<font color='red'>WARNING: LONG EXECUTION!</font>

In [74]:
from functools import partial

years = [2015, 2016, 2017, 2018, 2019, 2020]
speaker_qids = selected_country_speakers.id.to_list() 
output_list = []

for year in years:
    path_to_input = PATTERN_INPUT.format(year)
    
    # Process quote file
    output_name = process_compressed_json_file(path_to_input, 'data/country-speaker-quotes', year, partial(check_if_speaker_quote, speakers=speaker_qids))
    
    output_list.append(output_name)

print('\n\nOutput file names:')
for file_name in output_list:
    print(file_name)

Processing year 2015: 3189it [52:17,  1.02it/s]



File data/quotebank/quotes-2015.json.bz2 processed in 3140.3s



Processing year 2016: 2217it [34:26,  1.07it/s]



File data/quotebank/quotes-2016.json.bz2 processed in 2068.7s



Processing year 2017: 4962it [1:11:54,  1.15it/s]



File data/quotebank/quotes-2017.json.bz2 processed in 4318.2s



Processing year 2018: 194it [02:55,  1.11it/s]


KeyboardInterrupt: 